<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт--библиотек" data-toc-modified-id="Импорт--библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт  библиотек</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорт  библиотек

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

## Подготовка

In [2]:
toxic = pd.read_csv('/datasets/toxic_comments.csv')
toxic.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Первый столбик Unnamed: 0 просто дублирует индексы, поэтому от него нет особого смысла. Удалим его, чтобы не мешал

In [3]:
toxic = toxic.drop('Unnamed: 0', axis = 1)
toxic.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
toxic.shape

(159292, 2)

In [5]:
toxic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [6]:
toxic['toxic'].unique()

array([0, 1])

Можно оптимизировать столбец, сделав минимально возможный целый тип int8

In [7]:
toxic['toxic'] = toxic['toxic'].astype('int8')

Нужно проверить данные на дисбаланс, так как по первым столбцам можно заметить, что он возможен

In [8]:
len(toxic[toxic['toxic'] == 1])

16186

In [9]:
toxic

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Всего около 16 тысяч токсичных комментариев, а остальных почти в 10 раз больше. Это большая разница, которая повлияет на модель линейной регрессии, поэтому во время ее обучения устраним дисбаланс. Удалим строки с индексами > 50000, так как данных слишком много и даже достаточно простые моели будут очень долго обучаться

In [10]:
cut = toxic[toxic.index > 50000]
toxic = toxic.drop(cut.index)
toxic

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
49996,"Yes, I know that. Is there any reason why I s...",0
49997,It is about time we band RedDawn because he is...,0
49998,"(No, I'm not talking about the current instanc...",0
49999,"""\n\nMuch appreciated River. I'll name it afte...",0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(toxic.drop('toxic', axis = 1), toxic['toxic'], test_size=0.3, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(35000, 1) (15001, 1) (35000,) (15001,)


Получилось не ровно 35000 данных, по которым модели могут обучиться

Теперь можно очистить текст, чтобы будущие модельки могли обучиться

In [12]:
X_train['clear'] = X_train['text'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
X_test['clear'] = X_test['text'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
X_train.head()

,text,clear
40624,"yes, i've already said this. sentinel (MHS) ha...",yes i ve already said this sentinel MHS ha...
49426,"""\nI think the 1 million sales is total bullsh...",I think the million sales is total bullshi...
35734,"No problem, you fixed the article so there was...",No problem you fixed the article so there was...
41709,"""\n\nInstead, I've removed the reference to a ...",Instead I ve removed the reference to a pr...
31176,Kingdom Halls and Assembly Halls,Kingdom Halls and Assembly Halls


Лемматизируем очищенный текст

In [13]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

X_train['lim_clear'] = X_train['clear'].apply(lambda x: lemmatize_text(x))
X_test['lim_clear'] = X_test['clear'].apply(lambda x: lemmatize_text(x))
X_train.head()

,text,clear,lim_clear
40624,"yes, i've already said this. sentinel (MHS) ha...",yes i ve already said this sentinel MHS ha...,yes i ve already said this sentinel MHS had it...
49426,"""\nI think the 1 million sales is total bullsh...",I think the million sales is total bullshi...,I think the million sale is total bullshit tho...
35734,"No problem, you fixed the article so there was...",No problem you fixed the article so there was...,No problem you fixed the article so there wa n...
41709,"""\n\nInstead, I've removed the reference to a ...",Instead I ve removed the reference to a pr...,Instead I ve removed the reference to a previo...
31176,Kingdom Halls and Assembly Halls,Kingdom Halls and Assembly Halls,Kingdom Halls and Assembly Halls


In [14]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
X_train = X_train.drop(['text', 'clear'], axis = 1)
X_test = X_test.drop(['text', 'clear'], axis = 1)
corpus_train = X_train['lim_clear'].values
corpus_test = X_test['lim_clear'].values

count_tf_idf = TfidfVectorizer(stop_words = stop_words)
tfidf_train = count_tf_idf.fit_transform(corpus_train)
tfidf_test = count_tf_idf.transform(corpus_test)
print("Размер матрицы:", tfidf_train.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер матрицы: (35000, 67259)


Вот такая большая матрица у нас получилась. На этом подготовка данных для обучения модели завершается. Подведем итоги

У нас получились подготовленные данные. Изначально их было слишком много. Это могло привести к очень долгому обучению модели, поэтому пришлось оставить чуть больше трети данных. Это не должно повлиять на недообучение модели, так как около 40 тысяч члучаев ей хватит. После этого мы их разделили на тренировочные и тестовые данные. Последние будем использовать только для итоговой модели, а тренировочные данные были лемматизированы, очищены и приведены в читаемый для компьютера вид с помощью TF-IDF. Получилась матрица размером 45212 на 78342

## Обучение

На данном этапе обучим нескольно логистических моделей. Начнем с регрессии

In [15]:
param_grid = {'C': [0.01, 0.1, 1, 10]}
model = LogisticRegression(class_weight='balanced')
gs = GridSearchCV(model, param_grid, scoring = 'f1', cv = 3)
gs.fit(tfidf_train, y_train)
pred = gs.predict(tfidf_train)
best_model = gs.best_estimator_
print(best_model)
f1_1 = f1_score(y_train, pred)
print(f1_1)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, class_weight='balanced')
0.9554343488093655


С дисбалансом метрика была около 0.86, а без него уже окло 0.96 при параметре С = 10. Очень хороший результат. Можно проверить еще одну модель - дерево решений, вдруг она окажется лучше.

In [16]:
param_grid = {
    'max_depth': [3, 6, None],
    'min_samples_split': [10, 30, 70],
    'min_samples_leaf': [5, 15, 100]
}
model = DecisionTreeClassifier(random_state = 42)
gs = GridSearchCV(model, param_grid, scoring = 'f1', cv = 3)
gs.fit(tfidf_train, y_train)
pred = gs.predict(tfidf_train)
best_model = gs.best_estimator_
print(best_model)
f1_2 = f1_score(y_train, pred)
print(f1_2)

DecisionTreeClassifier(min_samples_leaf=5, min_samples_split=70,
                       random_state=42)
0.7765270898994446


При параметрах min_samples_leaf = 5, min_samples_split = 70 и max_depth = None получилась неплохая метрика около 0.78, но она меньше регрессии. Ну и метрики в 0.96 будет достаточно, поэтому перейдем к проверке логистической регрессии на тестовой выборке

In [18]:
model = LogisticRegression(class_weight='balanced', C = 10)
model.fit(tfidf_train, y_train)
pred = model.predict(tfidf_test)
f1 = f1_score(y_test, pred)
print(f1)

0.7561436672967863


на тестовой выборке метрика получилась 0.76. Это меньше, чем на тренировочной, но тоже неплохо. Можно подвести итоги шага

Мы обучили две модели классификации. Это логистическая регрессия и дерево решений. Их метрики F1 на тренировочных выборках получились такими: 0.96 и 0.78. Лучшей моделью оказалась логистическая регрессия, которая на тестовой выборке показала результат хуже: 0.76

## Выводы

Мы помогли интернет-магазину "Викишоп" побороться с оскорбительными комментариями в их новом сервисе, где можно оставлять отзывы и отвечать другим пользователям, из-за чего между ними может возникнуть конфликт. Для этого было решено создать модель, которая сможет различать комментарии на плохие и хорошие. Перед этим мы проделил такие шаги:
* Загрузили и подготовили данные. Заказчик предоставил нам их оченб много, поэтому мы оставили часть, чтобы модели могли достаточно быстро обучиться. После этого мы их разделили на тренировочные и тестовые данные. Последние использовали только для итоговой модели, а тренировочные данные были лемматизированы, очищены и приведены в читаемый для компьютера вид с помощью NtLK. Получилась матрица размером 40000 на 67259.
* Обучение моделей. Ключевой этап проекта, в котором мы выбирали между двумя обученными моделями. Это логистическая регрессия и дерево решений. Их метрики F1 на тренировочных выборках получились такими: 0.96 и 0.78.

Лучшей моделью оказалась логистическая регрессия, которая на тестовой выборке показала результат: 0.76
